### Import raw oligomer degradation data, normalize them, and plot them. Save the raw and normalized overlay plots for cathodic and anodic traces

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import os

# Parameters for folder path and peak detection
oligomer_names = ['Oligomer 1', 'Oligomer 2', 'Oligomer 3', 'Oligomer 4', 'Oligomer 5', 'Oligomer 6', 'Oligomer 7', 'Oligomer 8', 'Oligomer 9', 'Oligomer 10', 'Oligomer 14']  # Add all the oligomer names you want to process
base_folder_path = './Raw Electrochem Data'
processed_data_folder_path = './Processed Electrochem Data'  # Corrected to reference the base directory
prominence_value = 3e-6

def process_file(file_path, prominence_value, invert_current=False):
    # Processing logic remains unchanged
    data = pd.read_csv(file_path, delimiter='\t', skiprows=2, usecols=[2, 6], names=['Potential', 'Current'])
    if invert_current:
        data['Current'] = -data['Current']
    peaks, _ = find_peaks(data['Current'], prominence=prominence_value)
    peak_data = data.iloc[peaks]
    internal_standard_candidates = peak_data[
        (peak_data['Potential'] >= -0.15) & (peak_data['Potential'] <= 0.2) &
        (peak_data['Current'] >= 5e-6) & (peak_data['Current'] <= 8.5e-6)
    ]
    if not internal_standard_candidates.empty:
        internal_standard = internal_standard_candidates.iloc[0]
    else:
        raise ValueError("No internal standard candidates found within the specified range.")
    potential_shift = internal_standard['Potential']
    current_shift = internal_standard['Current']
    data['Normalized_Potential'] = data['Potential'] - potential_shift
    data['Normalized_Current'] = data['Current'] / current_shift
    return data, potential_shift, current_shift

def plot_and_save(legend_entries, plot_dir, oligomer_name, plot_title, file_suffix,
                  title_font_size=17, axis_label_font_size=24, axis_tick_font_size=24, legend_font_size=18):
    """
    Plot and save the normalized data with individual control over font sizes.
    """
    plt.figure(figsize=(12, 8), dpi=300)
    plt.rcParams.update({'font.size': axis_tick_font_size})

    # Define custom colors
    custom_colors = ['#4C4A59', '#E6B64E', '#0897B4', '#4CABA6', '#F2CDAC', '#13678A', 
                     '#F2CB05', '#45C4B0', '#0F91FF', '#F29325', '#F2D7AC', '#007172']

    # Filter out "Blank electrolyte" and non-numeric time points before processing
    unique_legend_entries = [
        (entry[0], tuple(entry[1]), tuple(entry[2]), entry[3])
        for entry in legend_entries if "blank electrolyte" not in entry[0].lower()
    ]

    # Filter and convert time points to integers
    filtered_legend_entries = []
    for entry in unique_legend_entries:
        try:
            time_label = int(entry[0].replace('min', ''))
            filtered_legend_entries.append((time_label, entry[1], entry[2], entry[3]))
        except ValueError:
            print(f"Skipping entry with non-numeric time point: {entry[0]}")
            continue

    filtered_legend_entries.sort(key=lambda x: x[0])
    
    num_time_points = min(len(filtered_legend_entries), 20)  # Limit to 20 time points for readability
    for i, entry in enumerate(filtered_legend_entries[:num_time_points]):
        time_label, potential, current, oligomer = entry
        plt.plot(potential, current, label=f'{time_label}', color=custom_colors[i % len(custom_colors)], linewidth=2)

    # Add legend and labels to the plot with specified font sizes
    plt.xlabel('Normalized Potential (V)', fontsize=axis_label_font_size)
    plt.ylabel('Normalized Current (Relative)', fontsize=axis_label_font_size)
    
    # Set legend with two columns, placed at the top left corner
    plt.legend(title="Minutes", fontsize=legend_font_size, title_fontsize=legend_font_size, 
               loc='upper right', ncol=1)

    # Turn off the grid
    plt.grid(False)

    # Set the x-axis limits
    plt.xlim(-0.2, 1.45)

    # Save the plot with the oligomer name included
    plot_path = os.path.join(plot_dir, f'{oligomer_name}_{file_suffix}.png')
    plt.savefig(plot_path, bbox_inches='tight')  # Remove white spaces
    print(f"Plot saved to {plot_path}")

    plt.show()

def plot_raw_and_save(raw_legend_entries, plot_dir, oligomer_name, plot_title, file_suffix,
                      title_font_size=17, axis_label_font_size=24, axis_tick_font_size=24, legend_font_size=18):
    """
    Plot and save the raw data with individual control over font sizes.
    """
    plt.figure(figsize=(12, 8), dpi=300)
    plt.rcParams.update({'font.size': axis_tick_font_size})

    # Define custom colors
    custom_colors = ['#4C4A59', '#E6B64E', '#0897B4', '#4CABA6', '#F2CDAC', '#13678A', 
                     '#F2CB05', '#45C4B0', '#0F91FF', '#F29325', '#F2D7AC', '#007172']

    # Filter out "Blank electrolyte" and non-numeric time points before processing
    unique_legend_entries = [
        (entry[0], tuple(entry[1]), tuple(entry[2]), entry[3])
        for entry in raw_legend_entries if "blank electrolyte" not in entry[0].lower()
    ]

    # Filter and convert time points to integers
    filtered_legend_entries = []
    for entry in unique_legend_entries:
        try:
            time_label = int(entry[0].replace('min', ''))
            filtered_legend_entries.append((time_label, entry[1], entry[2], entry[3]))
        except ValueError:
            print(f"Skipping entry with non-numeric time point: {entry[0]}")
            continue

    filtered_legend_entries.sort(key=lambda x: x[0])
    
    num_time_points = min(len(filtered_legend_entries), 20)  # Limit to 20 time points for readability
    for i, entry in enumerate(filtered_legend_entries[:num_time_points]):
        time_label, potential, current, oligomer = entry
        plt.plot(potential, current, label=f'{time_label}', color=custom_colors[i % len(custom_colors)], linewidth=2)

    # Add legend and labels to the plot with specified font sizes
    plt.xlabel('Potential (V)', fontsize=axis_label_font_size)
    plt.ylabel('Current (A)', fontsize=axis_label_font_size)
    
    # Set legend with two columns, placed at the top left corner
    plt.legend(title="Minutes", fontsize=legend_font_size, title_fontsize=legend_font_size, 
               loc='upper right', ncol=1)

    # Turn off the grid
    plt.grid(False)

    # Save the plot with the oligomer name included
    plot_path = os.path.join(plot_dir, f'{oligomer_name}_{file_suffix}.png')
    plt.savefig(plot_path, bbox_inches='tight')  # Remove white spaces
    print(f"Plot saved to {plot_path}")

    plt.show()

def run_and_save(base_folder_path, oligomer_names, prominence_value):
    """
    Run the processing and save results for all oligomers in the given list.
    """
    # Create the main directory for processed data
    os.makedirs(processed_data_folder_path, exist_ok=True)
    
    for oligomer_name in oligomer_names:
        oligomer_folder_path = os.path.join(processed_data_folder_path, oligomer_name)
        os.makedirs(oligomer_folder_path, exist_ok=True)

        anodic_result_dir = os.path.join(oligomer_folder_path, 'Normalized Anodic Trace Data')
        cathodic_result_dir = os.path.join(oligomer_folder_path, 'Normalized Cathodic Trace Data')
        plot_dir = os.path.join(oligomer_folder_path, 'Plots')

        # Ensure all necessary directories are created
        os.makedirs(anodic_result_dir, exist_ok=True)
        os.makedirs(cathodic_result_dir, exist_ok=True)
        os.makedirs(plot_dir, exist_ok=True)

        anodic_legend_entries = []
        cathodic_legend_entries = []
        raw_anodic_legend_entries = []
        raw_cathodic_legend_entries = []

        for root, _, files in os.walk(os.path.join(base_folder_path, oligomer_name)):
            for file in files:
                if file == 'DPV_Anodic_Trace.txt':
                    file_path = os.path.join(root, file)
                    time_point = os.path.basename(os.path.dirname(file_path))

                    if 'blank' in time_point.lower():
                        continue

                    try:
                        normalized_data, _, _ = process_file(file_path, prominence_value)
                        
                        anodic_legend_entries.append((time_point, list(normalized_data['Normalized_Potential']), list(normalized_data['Normalized_Current']), oligomer_name))
                        raw_anodic_legend_entries.append((time_point, list(normalized_data['Potential']), list(normalized_data['Current']), oligomer_name))

                        normalized_data_path = os.path.join(anodic_result_dir, f'{oligomer_name}_{time_point}_Anodic_Normalized.csv')
                        normalized_data.to_csv(normalized_data_path, index=False)

                        print(f"Processed and saved: {normalized_data_path}")

                    except Exception as e:
                        print(f"Error processing file {file_path}: {e}")

                elif file == 'DPV_Cathodic_Trace.txt':
                    file_path = os.path.join(root, file)
                    time_point = os.path.basename(os.path.dirname(file_path))

                    if 'blank' in time_point.lower():
                        continue

                    try:
                        normalized_data, _, _ = process_file(file_path, prominence_value, invert_current=True)
                        
                        cathodic_legend_entries.append((time_point, list(normalized_data['Normalized_Potential']), list(normalized_data['Normalized_Current']), oligomer_name))
                        raw_cathodic_legend_entries.append((time_point, list(normalized_data['Potential']), list(normalized_data['Current']), oligomer_name))

                        normalized_data_path = os.path.join(cathodic_result_dir, f'{oligomer_name}_{time_point}_Cathodic_Normalized.csv')
                        normalized_data.to_csv(normalized_data_path, index=False)

                        print(f"Processed and saved: {normalized_data_path}")

                    except Exception as e:
                        print(f"Error processing file {file_path}: {e}")

        if not anodic_legend_entries:
            print(f"No valid anodic oligomer data found for {oligomer_name} to plot.")
        else:
            plot_and_save(anodic_legend_entries, plot_dir, oligomer_name, f'Normalized DPV Anodic Traces for {oligomer_name}', 'Normalized_Anodic_trace_plot')
            plot_raw_and_save(raw_anodic_legend_entries, plot_dir, oligomer_name, f'Raw DPV Anodic Traces for {oligomer_name}', 'Raw_Anodic_trace_plot')
        
        if not cathodic_legend_entries:
            print(f"No valid cathodic oligomer data found for {oligomer_name} to plot.")
        else:
            plot_and_save(cathodic_legend_entries, plot_dir, oligomer_name, f'Normalized DPV Cathodic Traces for {oligomer_name}', 'Normalized_Cathodic_trace_plot')
            plot_raw_and_save(raw_cathodic_legend_entries, plot_dir, oligomer_name, f'Raw DPV Cathodic Traces for {oligomer_name}', 'Raw_Cathodic_trace_plot')

# Run the function to process files and save the results for all oligomers
run_and_save(base_folder_path, oligomer_names, prominence_value)


### Import normalized anodic and cathodic trace data for specified oligomers, perform curve fitting, and save the processed data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import os

# Define parameters
oligomers = ['Oligomer 1', 'Oligomer 2', 'Oligomer 3', 'Oligomer 4', 'Oligomer 5', 'Oligomer 6', 
             'Oligomer 7', 'Oligomer 8', 'Oligomer 9', 'Oligomer 10', 'Oligomer 14']
data_types = ['Anodic', 'Cathodic']
prominence_value = 0.0001
threshold_value = 0.1
min_width_value = 5
min_distance_value = 5
fit_range = 5
tolerance = 0.1  # Tolerance for peak matching
baseline_threshold = 0.02  # Threshold to consider a peak above baseline in 0 min data
dpi_value = 300  # Set higher DPI for better resolution

# Plot customization parameters
font_size_title = 23
font_size_legend = 18
font_size_axis_title = 23
font_size_axis_marks = 23
line_thickness = 3

# Define Gaussian function for curve fitting
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x - cen)**2 / (2 * wid**2))

# Function to detect peaks
def detect_peaks(data, prominence, threshold, min_width, min_distance):
    peaks, properties = find_peaks(data['Normalized_Current'], prominence=prominence, width=min_width, height=threshold, distance=min_distance)
    return peaks, properties

# Function to fit Gaussian curves around peaks iteratively
def fit_gaussians(data, peaks):
    fitted_peaks = []
    for peak in peaks:
        lower_bound = max(0, peak - fit_range)
        upper_bound = min(len(data) - 1, peak + fit_range)
        x_fit = data['Normalized_Potential'].iloc[lower_bound:upper_bound]
        y_fit = data['Normalized_Current'].iloc[lower_bound:upper_bound]

        popt = [y_fit.max(), x_fit[y_fit.idxmax()], 0.1]
        bounds = ([0, x_fit.min(), 0], [np.inf, x_fit.max(), np.inf])

        try:
            popt, _ = curve_fit(gaussian, x_fit, y_fit, p0=popt, bounds=bounds)
            fitted_peaks.append(popt)
        except RuntimeError:
            continue

    # Sort fitted peaks by their central potential (second element in popt)
    fitted_peaks.sort(key=lambda x: x[1])

    return fitted_peaks

# Function to process a single file and detect peaks
def process_file(file_path, prominence, threshold, min_width, min_distance):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None, None
    print(f"Processing file: {file_path}")
    data = pd.read_csv(file_path)
    peaks, properties = detect_peaks(data, prominence, threshold, min_width, min_distance)
    return data, peaks

# Function to plot data with detected peaks and Gaussian fits and save the plot
def plot_data_with_peaks_and_fits(data, peaks, fits, title, save_path):
    plt.figure(figsize=(12, 8), dpi=dpi_value)
    plt.plot(data['Normalized_Potential'], data['Normalized_Current'], label='Normalized Data', linewidth=line_thickness)
    plt.scatter(data['Normalized_Potential'].iloc[peaks], data['Normalized_Current'].iloc[peaks], color='red', label='Detected Peaks')

    # Sort fits by their central potential (x value)
    fits.sort(key=lambda x: x[1])

    for popt in fits:
        x_fit = np.linspace(data['Normalized_Potential'].min(), data['Normalized_Potential'].max(), 1000)
        y_fit = gaussian(x_fit, *popt)
        peak_x = popt[1]
        peak_y = popt[0]
        plt.plot(x_fit, y_fit, linestyle='--', label=f'(x={peak_x:.2f}, y={peak_y:.2f})', linewidth=line_thickness)

    plt.xlabel('Normalized Potential (V)', fontsize=font_size_axis_title)
    plt.ylabel('Normalized Current (Relative)', fontsize=font_size_axis_title)
    plt.title(title, fontsize=font_size_title)
    plt.legend(loc='best', fontsize=font_size_legend)
    plt.xticks(fontsize=font_size_axis_marks)
    plt.yticks(fontsize=font_size_axis_marks)
    plt.tight_layout()

    # Save the plot without displaying it in the notebook
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()  # Close the plot to free up memory

# Function to process all files in the specified directory
def process_directory(directory_path, oligomer_name, data_type, prominence, threshold, min_width, min_distance, output_folder, time_points):
    all_peak_data = []  # List to store peak data for all files

    # Process the 0 min data to identify the internal standard peak
    file_name = f'{oligomer_name}_0 min_{data_type}_Normalized.csv'
    file_path = os.path.join(directory_path, file_name)
    data_0min, peaks_0min = process_file(file_path, prominence, threshold, min_width, min_distance)
    if data_0min is None:
        return
    internal_standard_peak = [p for p in peaks_0min if abs(data_0min['Normalized_Potential'].iloc[p]) < tolerance and data_0min['Normalized_Current'].iloc[p] > 0.9]
    plot_data_with_peaks_and_fits(data_0min, internal_standard_peak, [], f'Normalized DPV Trace - 0 min', os.path.join(output_folder, f'{oligomer_name}_0 min_{data_type}_Normalized.png'))

    # Store peak data
    for p in internal_standard_peak:
        all_peak_data.append([oligomer_name, '0 min', data_0min['Normalized_Potential'].iloc[p], data_0min['Normalized_Current'].iloc[p]])

    # Process other time points to detect peaks and fit Gaussians
    for time_point in time_points[1:]:  # Exclude 0 min as it's already processed
        file_name = f'{oligomer_name}_{time_point}_{data_type}_Normalized.csv'
        file_path = os.path.join(directory_path, file_name)
        data, peaks = process_file(file_path, prominence, threshold, min_width, min_distance)
        if data is None:
            continue

        current_fits = []
        for peak in peaks:
            if data['Normalized_Current'].iloc[peak] > baseline_threshold:
                lower_bound = max(0, peak - fit_range)
                upper_bound = min(len(data) - 1, peak + fit_range)
                x_fit = data['Normalized_Potential'].iloc[lower_bound:upper_bound]
                y_fit = data['Normalized_Current'].iloc[lower_bound:upper_bound]
                try:
                    p0 = [y_fit.max(), x_fit[y_fit.idxmax()], 0.1]
                    bounds = ([0, x_fit.min(), 0], [np.inf, x_fit.max(), np.inf])
                    popt, _ = curve_fit(gaussian, x_fit, y_fit, p0=p0, bounds=bounds)
                    current_fits.append(popt)
                except RuntimeError:
                    continue

        plot_title = f'Normalized DPV Trace with Detected Peaks - {time_point}'
        plot_path = os.path.join(output_folder, f'{oligomer_name}_{time_point}_{data_type}_Normalized.png')
        plot_data_with_peaks_and_fits(data, peaks, current_fits, plot_title, plot_path)

        # Store peak data
        for fit in current_fits:
            all_peak_data.append([oligomer_name, time_point, fit[1], fit[0]])

    # Save all peak data to an Excel file in ascending order of time points
    peak_data_df = pd.DataFrame(all_peak_data, columns=['Oligomer Name', 'Time Point', 'Potential (V)', 'Current (Relative)'])
    peak_data_df['Time Point'] = pd.Categorical(peak_data_df['Time Point'], categories=time_points, ordered=True)
    peak_data_df = peak_data_df.sort_values('Time Point')
    peak_data_df.to_excel(os.path.join(output_folder, f'{oligomer_name}_peak_data.xlsx'), index=False)

# Main processing loop for all oligomers and data types
base_processed_folder_path = './Processed Electrochem Data'

for oligomer in oligomers:
    for data_type in data_types:
        # Update the folder path to read normalized data from the "Processed Electrochem Data" folder
        folder_path = f'{base_processed_folder_path}/{oligomer}/Normalized {data_type} Trace Data/'
        # Update the output folder to save processed data
        output_folder = os.path.join(base_processed_folder_path, oligomer, f'{data_type} Curve Fitted Data')
        os.makedirs(output_folder, exist_ok=True)
        time_points = ['0 min', '10 min', '20 min', '30 min', '40 min', '50 min', '65 min', '80 min', '95 min', '110 min', '130 min', '150 min']
        process_directory(folder_path, oligomer, data_type, prominence_value, threshold_value, min_width_value, min_distance_value, output_folder, time_points)


##### This code processes the peak current data from curve fitting for each oligomer (both anodic and cathodic), and filters the peak values for S1, S2, S3, and S4, and also makes an assignment of Monomers (M1, M2, M3, M4) to the S values. Then it consolidates this information in 4 excel files, two containing monomer assignments for each oligomer, and two containing S1, S2, S3, and S4 values for all oligomers for further analysis

In [ ]:
import pandas as pd
import os
from IPython.display import display

# Define the list of oligomers to process
oligomers = ['Oligomer 1', 'Oligomer 2', 'Oligomer 3', 'Oligomer 4', 'Oligomer 5', 'Oligomer 6', 
             'Oligomer 7', 'Oligomer 8', 'Oligomer 9', 'Oligomer 10', 'Oligomer 14']

# Base directory where the processed peak data is stored
base_directory = './Processed Electrochem Data'

# Define time points and the potential ranges for the monomers
time_points = [0, 10, 20, 30, 40, 50, 65, 80, 95, 110, 130, 150]
monomer_ranges = {
    'M1': (0.60, 0.67),
    'M2': (0.75, 0.85),
    'M3': (0.90, 1.0),
    'M4': (1.15, 1.30)
}

# Initialize lists to store assignment and S values data for all oligomers
cathodic_assignments = []
anodic_assignments = []
cathodic_values = []
anodic_values = []

# Function to process the data for a given type (Cathodic/Anodic)
def process_curve_data(file_path, oligomer_name, curve_type):
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name='Sheet1')

    # Print the column names to check for discrepancies
    print(f"Processing: {oligomer_name} ({curve_type})")
    print("Column names in the DataFrame:", df.columns)

    # Initialize the list to store data for the new DataFrame
    data = []

    # Get the unique oligomer name
    oligomer_name = df['Oligomer Name'].unique()[0]

    # Initialize a list to hold the first detection information for each monomer
    first_detection = []

    # Define the column names
    potential_column = 'Potential (V)'
    current_column = 'Current (Relative)'

    # Step 1: Find the first detection time and current value for each monomer
    for time in time_points:
        # Filter data for the current time point
        time_data = df[df['Time Point'] == f'{time} min']
        
        for _, row in time_data.iterrows():
            potential = row[potential_column]
            current = row[current_column]
            for monomer, (low, high) in monomer_ranges.items():
                if low <= potential <= high:
                    first_detection.append((monomer, time, current))

    # Step 2: Sort the first detection list based on time and current value
    first_detection.sort(key=lambda x: (x[1], -x[2]))

    # Step 3: Assign monomers to S1, S2, S3, and S4 based on the sorted list
    assignments = {}
    assigned_monomers = set()
    for idx, (monomer, _, _) in enumerate(first_detection):
        if monomer not in assigned_monomers:
            assignments[f'S{len(assignments) + 1}'] = monomer
            assigned_monomers.add(monomer)
            if len(assignments) == 4:
                break

    # Step 4: Process the data to assign current values to S1, S2, S3, S4
    for time in time_points:
        # Filter data for the current time point
        time_data = df[df['Time Point'] == f'{time} min']
        
        # Initialize the peak current values for S1, S2, S3, S4 as 0
        s_values = {key: 0 for key in ['S1', 'S2', 'S3', 'S4']}
        
        for _, row in time_data.iterrows():
            potential = row[potential_column]
            current = row[current_column]
            for s_label, monomer in assignments.items():
                low, high = monomer_ranges[monomer]
                if low <= potential <= high:
                    s_values[s_label] = current
        
        # Append the row data to the list
        data.append({
            'Oligomer': oligomer_name,
            'Time': time,
            **s_values
        })

    # Convert the list to a DataFrame
    output_df = pd.DataFrame(data)

    # Rename the columns to include the monomer assignments
    for s_label, monomer in assignments.items():
        output_df.rename(columns={s_label: f"{s_label}({monomer})"}, inplace=True)

    # Construct the output file path for processed data
    output_directory = os.path.join(base_directory, oligomer_name, 'concentration-time data')
    os.makedirs(output_directory, exist_ok=True)
    output_file_name = f"Processed_{curve_type}_peak_data.xlsx"
    output_file_path = os.path.join(output_directory, output_file_name)

    # Save the new DataFrame to an Excel file in the specified directory
    output_df.to_excel(output_file_path, index=False)

    # Display the new DataFrame
    display(output_df.head())

    print(f"Processed file saved as: {output_file_name}\n")

    # Append the assignments to the respective list based on the curve type
    for s_label, monomer in assignments.items():
        if curve_type == 'Cathodic':
            cathodic_assignments.append({
                'Oligomer': oligomer_name,
                'Assignment': s_label,
                'Monomer': monomer
            })
        elif curve_type == 'Anodic':
            anodic_assignments.append({
                'Oligomer': oligomer_name,
                'Assignment': s_label,
                'Monomer': monomer
            })

    # Adjust the S1, S2, S3, S4 values for missing data across all time points
    for s_label in ['S1', 'S2', 'S3', 'S4']:
        if all(record[s_label] == 0 for record in data):
            for record in data:
                record[s_label] = float('nan')

    # Append the S1, S2, S3, S4 values to the respective list based on the curve type
    if curve_type == 'Cathodic':
        cathodic_values.extend(data)
    elif curve_type == 'Anodic':
        anodic_values.extend(data)

# Process each oligomer in the list for Cathodic data first and then Anodic data
for oligomer in oligomers:
    # Process Cathodic data
    cathodic_file_path = os.path.join(base_directory, oligomer, 'Cathodic Curve Fitted Data', f'{oligomer}_peak_data.xlsx')
    
    if os.path.exists(cathodic_file_path):
        process_curve_data(cathodic_file_path, oligomer, 'Cathodic')
    else:
        print(f"Cathodic file not found for {oligomer}, skipping.")

    # Process Anodic data
    anodic_file_path = os.path.join(base_directory, oligomer, 'Anodic Curve Fitted Data', f'{oligomer}_peak_data.xlsx')
    
    if os.path.exists(anodic_file_path):
        process_curve_data(anodic_file_path, oligomer, 'Anodic')
    else:
        print(f"Anodic file not found for {oligomer}, skipping.")

# After processing all oligomers, save the consolidated assignments to separate Excel files
cathodic_assignments_df = pd.DataFrame(cathodic_assignments)
anodic_assignments_df = pd.DataFrame(anodic_assignments)

cathodic_values_df = pd.DataFrame(cathodic_values)
anodic_values_df = pd.DataFrame(anodic_values)

# Specify the file paths with the desired names
cathodic_assignments_file_path = os.path.join(base_directory, 'Monomer_Assignments_Cathodic.xlsx')
anodic_assignments_file_path = os.path.join(base_directory, 'Monomer_Assignments_Anodic.xlsx')

cathodic_values_file_path = os.path.join(base_directory, 'Combined_Cathodic_Experimental_Peak_Currents.xlsx')
anodic_values_file_path = os.path.join(base_directory, 'Combined_Anodic_Experimental_Peak_Currents.xlsx')

# Save the DataFrames to the specified file paths
cathodic_assignments_df.to_excel(cathodic_assignments_file_path, index=False)
anodic_assignments_df.to_excel(anodic_assignments_file_path, index=False)

cathodic_values_df.to_excel(cathodic_values_file_path, index=False)
anodic_values_df.to_excel(anodic_values_file_path, index=False)

print(f"Cathodic assignments file saved as: {cathodic_assignments_file_path}")
print(f"Anodic assignments file saved as: {anodic_assignments_file_path}")
print(f"Cathodic values file saved as: {cathodic_values_file_path}")
print(f"Anodic values file saved as: {anodic_values_file_path}")

### Import experimental peak current data, perform curve fit, and normalize the curve-fitted data points, and save them on an excel file for PCA Analysis

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
from scipy.interpolate import make_interp_spline

# Define file paths for cathodic and anodic data
cathodic_file_path = './Processed Electrochem Data/Combined_Cathodic_Experimental_Peak_Currents.xlsx'
anodic_file_path = './Processed Electrochem Data/Combined_Anodic_Experimental_Peak_Currents.xlsx'

# Define file paths for monomer assignments
cathodic_assignment_path = './Processed Electrochem Data/Monomer_Assignments_Cathodic.xlsx'
anodic_assignment_path = './Processed Electrochem Data/Monomer_Assignments_Anodic.xlsx'

# Load monomer assignments
cathodic_assignments = pd.read_excel(cathodic_assignment_path)
anodic_assignments = pd.read_excel(anodic_assignment_path)

# Pivot the assignments DataFrame
cathodic_assignments = cathodic_assignments.pivot(index='Oligomer', columns='Assignment', values='Monomer')
anodic_assignments = anodic_assignments.pivot(index='Oligomer', columns='Assignment', values='Monomer')

# Define the base directory for PCA Analysis
pca_analysis_dir = './PCA Analysis'

def process_oligomer_data(data_type, file_path, assignments):
    # Load the known oligomer data from the Excel file
    known_df = pd.read_excel(file_path)

    # Define the intensity columns
    intensity_columns = ['S1', 'S2', 'S3', 'S4']

    # Filter out rows where all intensity values are NaN
    known_df = known_df.dropna(subset=intensity_columns, how='all')

    # Ensure 'Oligomer' column exists
    if 'Oligomer' not in known_df.columns:
        raise KeyError("'Oligomer' column not found in the DataFrame")

    # Define main directory based on data type
    main_dir = os.path.join(pca_analysis_dir, data_type.capitalize())

    # Define directories for saving plots and data
    plots_dir = os.path.join(main_dir, 'Fitted plots')
    excel_dir = os.path.join(main_dir, 'Fitted Peak Currents')
    normalized_data_dir = os.path.join(main_dir, 'Fitted&Normalized Peak Currents')
    normalized_plots_dir = os.path.join(main_dir, 'Fitted and Normalized Plots')

    # Create directories if they do not exist
    for dir_path in [plots_dir, excel_dir, normalized_data_dir, normalized_plots_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Define a modified logistic fitting function
    def modified_logistic(x, L, k, x_0):
        return L / (1 + np.exp(-k * (x - x_0))) - L / (1 + np.exp(k * x_0))

    # Plot settings
    line_thickness = 4
    axis_label_font_size = 28
    axis_value_font_size = 28
    legend_font_size = 28
    marker_size = 30  # Marker size for data points

    # Monomer colors
    monomer_colors = {
        'M1': '#F23030',
        'M2': '#F2A71B',
        'M3': '#A1C7E0',
        'M4': '#009688'
    }

    # Initialize DataFrames to store fitted and normalized data
    all_fitted_data_df = pd.DataFrame()
    all_normalized_data_df = pd.DataFrame()

    # Process each oligomer
    for oligomer in known_df['Oligomer'].unique():
        oligomer_data = known_df[known_df['Oligomer'] == oligomer].dropna(subset=['Time'])
        
        # Check if the assignment is available for this oligomer
        if oligomer not in assignments.index:
            print(f"No assignment found for oligomer {oligomer}")
            continue

        # Get the assignment for the current oligomer
        assignment = assignments.loc[oligomer]

        # Create a new figure for the fitted data
        plt.figure(figsize=(12, 8))
        fitted_data_df = pd.DataFrame({'Time': oligomer_data['Time'].sort_values().unique()})
        max_y_value = 0
        
        # Fit and plot each intensity column
        for col in intensity_columns:
            if col in assignment and oligomer_data[col].notna().any():
                oligomer_data = oligomer_data.sort_values('Time')
                time = oligomer_data['Time']
                intensity = oligomer_data[col]
                monomer = assignment[col]
                
                if len(time) > 3:
                    try:
                        p0 = [max(intensity), 1, np.median(time)]
                        bounds = (0, [max(intensity) * 1.5, 10, time.max()])
                        popt, _ = curve_fit(modified_logistic, time, intensity, p0=p0, bounds=bounds, maxfev=5000)
                        
                        time_smooth = np.linspace(time.min(), time.max(), 300)
                        intensity_smooth = modified_logistic(time_smooth, *popt)
                        
                        plt.plot(time_smooth, intensity_smooth, label=f'{col}={monomer}', linewidth=line_thickness, color=monomer_colors[monomer])
                        plt.scatter(time, intensity, marker='o', s=marker_size, color=monomer_colors[monomer])
                        
                        fitted_data_df[col] = modified_logistic(fitted_data_df['Time'], *popt)
                        max_y_value = max(max_y_value, intensity.max(), intensity_smooth.max())
                    except RuntimeError as e:
                        print(f"Could not fit data for oligomer {oligomer} and intensity {col}: {e}")
                else:
                    print(f"Insufficient data to fit for oligomer {oligomer} and intensity {col}")

        # Adjust the legend to only include the fitted plot lines
        plt.xlabel('Time (min)', fontsize=axis_label_font_size)
        plt.ylabel('Relative Concentration', fontsize=axis_label_font_size)
        plt.legend(title=f'{oligomer}', fontsize=legend_font_size, title_fontsize=legend_font_size)  # Set legend title as the oligomer name
        plt.xlim(0, 160)
        plt.ylim(-0.05, max_y_value * 1.1)
        plt.xticks(fontsize=axis_value_font_size)
        plt.yticks(fontsize=axis_value_font_size)

        # Adjust layout and remove white space around the plot
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f'{oligomer}_intensity_over_time.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        fitted_data_df.insert(0, 'Oligomer', oligomer)
        all_fitted_data_df = pd.concat([all_fitted_data_df, fitted_data_df], ignore_index=True)
        
        print(f"Processed and saved plots and data for oligomer {oligomer}")

    # Save all fitted data to an Excel file
    all_fitted_data_path = os.path.join(excel_dir, 'all_oligomers_fitted_data.xlsx')
    all_fitted_data_df.to_excel(all_fitted_data_path, index=False)

    # Normalize and plot the fitted data
    for oligomer in all_fitted_data_df['Oligomer'].unique():
        oligomer_data = all_fitted_data_df[all_fitted_data_df['Oligomer'] == oligomer]
        present_intensity_columns = [col for col in intensity_columns if col in oligomer_data.columns]
        max_value = oligomer_data[present_intensity_columns].max().max()
        
        normalized_df = oligomer_data.copy()
        if max_value != 0:
            normalized_df[present_intensity_columns] = oligomer_data[present_intensity_columns] / max_value
        
        all_normalized_data_df = pd.concat([all_normalized_data_df, normalized_df], ignore_index=True)
        
        plt.figure(figsize=(12, 8))
        max_y_value_normalized = 0
        
        # Retrieve the assignment for the current oligomer
        assignment = assignments.loc[oligomer]

        for col in present_intensity_columns:
            if col in assignment and normalized_df[col].notna().any():
                time = normalized_df['Time']
                intensity = normalized_df[col]
                monomer = assignment[col]
                
                time_smooth = np.linspace(time.min(), time.max(), 300)
                spl = make_interp_spline(time, intensity, k=3)
                intensity_smooth = spl(time_smooth)
                
                plt.plot(time_smooth, intensity_smooth, label=f'{col}={monomer}', linewidth=line_thickness, color=monomer_colors[monomer])
                plt.scatter(time, intensity, marker='o', s=marker_size, color=monomer_colors[monomer])
                
                max_y_value_normalized = max(max_y_value_normalized, intensity.max())
        
        plt.xlabel('Time (min)', fontsize=axis_label_font_size)
        plt.ylabel('Relative Concentration', fontsize=axis_label_font_size)
        plt.legend(title=f'{oligomer}', fontsize=legend_font_size, title_fontsize=legend_font_size)  # Set legend title as the oligomer name
        plt.xlim(0, 160)
        plt.ylim(-0.05, max_y_value_normalized * 1.1)
        plt.xticks(fontsize=axis_value_font_size)
        plt.yticks(fontsize=axis_value_font_size)

        # Adjust layout and remove white space around the plot
        plt.tight_layout()
        plt.savefig(os.path.join(normalized_plots_dir, f'{oligomer}_fitted_and_normalized_intensity_over_time.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Processed and saved normalized plots and data for oligomer {oligomer}")

    # Save all normalized data to an Excel file
    all_normalized_data_path = os.path.join(normalized_data_dir, 'all_oligomers_fitted_and_normalized_fitted_data.xlsx')
    all_normalized_data_df.to_excel(all_normalized_data_path, index=False)

    print(f"All {data_type} plots and data files have been saved successfully.")

# Process both cathodic and anodic data
process_oligomer_data('cathodic', cathodic_file_path, cathodic_assignments)
process_oligomer_data('anodic', anodic_file_path, anodic_assignments)


### Perform PCA Analysis on known standards and unknown oligomer data to determine the sequence type of unknown oligomers. Perform sequence matching

In [ ]:
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
import numpy as np
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Define file paths
known_file = './PCA Analysis/PCA_Standard_Data.xlsx'
unknown_file = './PCA Analysis/Cathodic/Fitted&Normalized Peak Currents/all_oligomers_fitted_and_normalized_fitted_data.xlsx'
intensity_columns = ['S1', 'S2', 'S3', 'S4']

# Define the base output directory for PCA Analysis
pca_analysis_dir = './PCA Analysis'

# Ensure that the base output directory exists
os.makedirs(pca_analysis_dir, exist_ok=True)

# Average conversion factors
conversion_factors = {
    'S1': 0.929,
    'S2': 0.910,
    'S3': 0.690
}

offsets = {
    'S1': 0,
    'S2': 0,
    'S3': 0
}

def load_and_prepare_data(known_file, unknown_file, intensity_columns):
    # Load the known and unknown data
    known_df = pd.read_excel(known_file)
    unknown_df = pd.read_excel(unknown_file)

    # Ensure all required columns exist, fill missing columns with NaN
    for col in intensity_columns:
        if col not in known_df.columns:
            known_df[col] = np.nan
        if col not in unknown_df.columns:
            unknown_df[col] = np.nan

    # Apply conversion factors to the unknown data
    for col in intensity_columns[:-1]:  # Exclude 'S4' which has no conversion factor
        if col in conversion_factors:
            unknown_df[col] = unknown_df[col] * conversion_factors[col] + offsets[col]

    # Add intensity count and presence indicators
    for df in [known_df, unknown_df]:
        df['intensity_count'] = df[intensity_columns].notna().sum(axis=1)
        for col in intensity_columns:
            df[f'{col}_present'] = df[col].notna().astype(int)

    # Aggregate data by 'Oligomer'
    def aggregate_data(df):
        return df.groupby('Oligomer').agg({
            **{col: 'mean' for col in intensity_columns},
            **{f'{col}_present': 'mean' for col in intensity_columns},
            'intensity_count': 'max'
        }).reset_index()

    grouped_known_df = aggregate_data(known_df)
    grouped_unknown_df = aggregate_data(unknown_df)

    # Fill missing intensity values with zero
    grouped_known_df[intensity_columns] = grouped_known_df[intensity_columns].fillna(0)
    grouped_unknown_df[intensity_columns] = grouped_unknown_df[intensity_columns].fillna(0)

    return grouped_known_df, grouped_unknown_df

def perform_pca(data, n_components=5):
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data)
    
    # Perform PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(X_scaled)
    
    return principal_components, pca, scaler

def find_best_matches(known_df, unknown_df, principal_components_known, principal_components_unknown):
    matches = []

    for _, unknown_row in unknown_df.iterrows():
        count = unknown_row['intensity_count']
        filtered_known = known_df[known_df['intensity_count'] == count]
        
        # Check if filtered_known is empty
        if filtered_known.empty:
            print(f"No known oligomers with intensity count {count} found for unknown oligomer {unknown_row['Oligomer']}. Skipping this oligomer.")
            continue

        distances = filtered_known.apply(
            lambda row: euclidean(row[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']], unknown_row[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']]),
            axis=1
        )
        
        # Find the best match
        best_match_idx = distances.idxmin()
        best_match = filtered_known.loc[best_match_idx]
        
        # Find the second-best match, if available
        if len(distances) > 1:
            second_best_match_idx = distances.drop(best_match_idx).idxmin()
            second_best_match = filtered_known.loc[second_best_match_idx]
            matches.append({
                'Unknown_Oligomer': unknown_row['Oligomer'],
                'Best_Match_Oligomer': best_match['Oligomer'],
                'Second_Best_Match_Oligomer': second_best_match['Oligomer'],
                'Best_Match_Distance': distances.min(),
                'Second_Best_Match_Distance': distances.drop(best_match_idx).min()
            })
        else:
            # Only one match available, set second-best to None
            matches.append({
                'Unknown_Oligomer': unknown_row['Oligomer'],
                'Best_Match_Oligomer': best_match['Oligomer'],
                'Second_Best_Match_Oligomer': None,
                'Best_Match_Distance': distances.min(),
                'Second_Best_Match_Distance': None
            })

    return pd.DataFrame(matches)

def create_3d_plot(known_df, unknown_df, show_unknown=True):
    # Combine known and unknown data
    known_df['Type'] = known_df['intensity_count'].apply(lambda x: f'Count_{int(x)}')
    if show_unknown:
        unknown_df['Type'] = 'Unknown'
        combined_df = pd.concat([known_df, unknown_df], ignore_index=True)
    else:
        combined_df = known_df

    # Define custom color palette for different intensity counts
    color_map = {
        'Count_1': '#636EFA',  # Color for intensity count 1
        'Count_2': '#009688',  # Color for intensity count 2
        'Count_3': '#FFA15A',  # Color for intensity count 3
        'Count_4': '#AB63FA',  # Color for intensity count 4
        'Unknown': '#EF553B'   # Color for unknown oligomers
    }

    # Initialize 3D scatter plot with individual traces for each type to ensure labels remain visible
    fig = go.Figure()

    # Add each type as a separate trace to control label visibility and color
    for type_value in combined_df['Type'].unique():
        subset = combined_df[combined_df['Type'] == type_value]
        fig.add_trace(
            go.Scatter3d(
                x=subset['PC1'], y=subset['PC2'], z=subset['PC3'],
                mode='markers+text',  # Show both markers and text
                marker=dict(size=8, color=color_map[type_value], line=dict(width=0)),
                text=subset['Oligomer'],  # Display oligomer name
                textposition="top center",  # Position text above the marker
                textfont=dict(size=20),  # Adjust font size for readability
                name=type_value  # Legend label
            )
        )

    # Add drop lines from each point to a fixed z-value (e.g., -2.0)
    for i, row in combined_df.iterrows():
        fig.add_trace(
            go.Scatter3d(
                x=[row['PC1'], row['PC1']],
                y=[row['PC2'], row['PC2']],
                z=[row['PC3'], -2.0],
                mode='lines',
                line=dict(color=color_map[row['Type']], width=6),
                showlegend=False
            )
        )

    # Update layout for publication quality
    fig.update_layout(
        scene=dict(
            xaxis=dict(title=dict(font=dict(size=25), text='Principal Component 1'), tickfont=dict(size=16)),
            yaxis=dict(title=dict(font=dict(size=25), text='Principal Component 2'), tickfont=dict(size=16)),
            zaxis=dict(title=dict(font=dict(size=25), text='Principal Component 3'), tickfont=dict(size=16)),
            aspectmode='cube',
            camera_eye=dict(x=1.25, y=1.25, z=1.25)
        ),
        legend=dict(title=dict(text='Intensity Count', font=dict(size=23))),
        font=dict(family="Arial", size=23),
        margin=dict(l=0, r=0, b=0, t=30),
        template='plotly_white',
        title='3D PCA of Oligomer Data with Known and Unknown Oligomers' if show_unknown else '3D PCA of Oligomer Data with Known Oligomers'
    )

    # Define output path for the HTML plot
    plot_path = os.path.join(pca_analysis_dir, '3D_PCA_Oligomer_Plot_Publication_Quality.html')
    # Save and show the plot
    pio.write_html(fig, file=plot_path, auto_open=True)
    fig.show()

def plot_variance(explained_variance_ratio):
    plt.figure(figsize=(12, 8))
    plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.5, align='center')
    plt.ylabel('Explained variance ratio', fontsize=25)
    plt.xlabel('Principal components', fontsize=25)
    plt.title('Variance Explained by Each Principal Component', fontsize=23)

    # Annotate each bar with the variance ratio
    for i, v in enumerate(explained_variance_ratio):
        plt.text(i + 1, v, f"{v:.2f}", ha='center', va='bottom', fontsize=23)

    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)

    # Define output path for variance plot
    variance_plot_path = os.path.join(pca_analysis_dir, 'Variance_Explained_by_Principal_Components.png')
    plt.savefig(variance_plot_path, bbox_inches='tight')
    plt.show()

def main():
    # Load and prepare data
    grouped_known_df, grouped_unknown_df = load_and_prepare_data(known_file, unknown_file, intensity_columns)

    # Perform PCA on known data
    principal_components_known, pca, scaler = perform_pca(grouped_known_df[intensity_columns + [f'{col}_present' for col in intensity_columns]])

    # Transform unknown data using the same PCA model
    unknown_X_scaled = scaler.transform(grouped_unknown_df[intensity_columns + [f'{col}_present' for col in intensity_columns]])
    principal_components_unknown = pca.transform(unknown_X_scaled)

    # Create DataFrame with principal components
    known_principal_df = pd.DataFrame(data=principal_components_known, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
    known_result_df = pd.concat([grouped_known_df[['Oligomer', 'intensity_count']], known_principal_df], axis=1)

    unknown_principal_df = pd.DataFrame(data=principal_components_unknown, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
    unknown_result_df = pd.concat([grouped_unknown_df[['Oligomer', 'intensity_count']], unknown_principal_df], axis=1)

    # Find best matches
    matches_df = find_best_matches(known_result_df, unknown_result_df, principal_components_known, principal_components_unknown)

    # Save matches to Excel
    matches_file_path = os.path.join(pca_analysis_dir, 'oligomer_matches.xlsx')
    matches_df.to_excel(matches_file_path, index=False)

    # Print matches
    print("Matches DataFrame:\n", matches_df)

    # User can decide whether to show unknown oligomers in the 3D plot
    show_unknown = False  # Set to False if you do not want to display unknown oligomers
    create_3d_plot(known_result_df, unknown_result_df, show_unknown=show_unknown)

    # Plot variance
    plot_variance(pca.explained_variance_ratio_)

if __name__ == "__main__":
    main()


### Construct the exact sequence of the analyzed oligomers

In [ ]:
import pandas as pd

# File paths for the required Excel files
monomer_assignments_path = './Processed Electrochem Data/Monomer_Assignments_Cathodic.xlsx'
oligomer_matches_path = './PCA Analysis/oligomer_matches.xlsx'
sequences_path = './PCA Analysis/Sequences.xlsx'

# Load the Excel files
monomer_assignments = pd.read_excel(monomer_assignments_path)
oligomer_matches = pd.read_excel(oligomer_matches_path)
sequences = pd.read_excel(sequences_path)

# Function to reconstruct the exact sequence by replacing placeholders with actual monomers
def reconstruct_sequence(sequence_row, oligomer_name, monomer_assignments):
    # Extract the placeholder sequence for the oligomer (e.g., S1, S2, S3, S4)
    placeholder_sequence = [sequence_row['First'], sequence_row['Second'], sequence_row['Third'], sequence_row['Fourth']]
    
    # Get the monomer assignments for this oligomer
    oligomer_monomer_assignments = monomer_assignments[monomer_assignments['Oligomer'] == oligomer_name]
    
    # Replace placeholders with actual monomers
    reconstructed_sequence = [
        oligomer_monomer_assignments[oligomer_monomer_assignments['Assignment'] == placeholder]['Monomer'].values[0]
        for placeholder in placeholder_sequence
    ]
    
    return reconstructed_sequence

# Iterate over oligomer matches and reconstruct sequences for the best matches
reconstructed_sequences = []

for _, row in oligomer_matches.iterrows():
    oligomer_name = row['Unknown_Oligomer']
    
    # Get the sequence name for the best match
    best_match_seq_name = row['Best_Match_Oligomer']
    
    # Get the actual sequence using the sequence name
    best_match_sequence_row = sequences[sequences['Sequence'] == best_match_seq_name].iloc[0]
    
    # Reconstruct the sequence for the best match
    best_match_reconstructed = reconstruct_sequence(best_match_sequence_row, oligomer_name, monomer_assignments)
    
    # Save the results with separate columns for First, Second, Third, and Fourth
    reconstructed_sequences.append({
        'Unknown_Oligomer': oligomer_name,
        'Best_Match_Oligomer': best_match_seq_name,
        'First': best_match_reconstructed[0],
        'Second': best_match_reconstructed[1],
        'Third': best_match_reconstructed[2],
        'Fourth': best_match_reconstructed[3]
    })
    
    # Print the best match reconstructed sequence
    print(f"Best match reconstructed sequence for {oligomer_name}: {best_match_reconstructed}")

# Convert the results into a dataframe
reconstructed_sequences_df = pd.DataFrame(reconstructed_sequences)

# Display the reconstructed sequences
reconstructed_sequences_df.head()

# Save the final reconstructed sequences to an Excel file in the current directory
output_file_path = 'best_match_reconstructed_sequences.xlsx'
reconstructed_sequences_df.to_excel(output_file_path, index=False)

print(f"Best match reconstructed sequences saved to {output_file_path}")


### Decode the ASCII Characters stored by each oligomer

In [ ]:
import pandas as pd

# File paths for the required Excel files
assigned_information_path = './assigned_information.xlsx'
reconstructed_sequences_path = './best_match_reconstructed_sequences.xlsx'

# Load the Excel files
assigned_information = pd.read_excel(assigned_information_path)
reconstructed_sequences_df = pd.read_excel(reconstructed_sequences_path)

# Function to match the best match sequence to the possible sequences and find the corresponding ASCII character
def find_ascii_for_sequence(row, assigned_information):
    # Compare the four monomers with the assigned_information monomers
    matched_row = assigned_information[
        (assigned_information['First'] == row['First']) &
        (assigned_information['Second'] == row['Second']) &
        (assigned_information['Third'] == row['Third']) &
        (assigned_information['Fourth'] == row['Fourth'])
    ]
    
    if not matched_row.empty:
        return matched_row['Encoded ASCII Character'].values[0]  # Return the first match found
    return None  # Return None if no match is found

# Create a new column to store the ASCII character for each best match reconstructed sequence
reconstructed_sequences_df['Encoded_ASCII_Character'] = reconstructed_sequences_df.apply(
    lambda row: find_ascii_for_sequence(row, assigned_information), axis=1
)

# Display the results
print(reconstructed_sequences_df[['Unknown_Oligomer', 'First', 'Second', 'Third', 'Fourth', 'Encoded_ASCII_Character']])

# Save the updated dataframe to a new Excel file
output_file_path = './reconstructed_sequences_with_ascii.xlsx'
reconstructed_sequences_df.to_excel(output_file_path, index=False)

print(f"Reconstructed sequences with ASCII characters saved to {output_file_path}")
